In [6]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import KFold

# --- Utility Functions ---
# --- Utility Functions ---

def load_data(file_path):
    """Loads and tokenizes the POS-tagged sentences in word_TAG format."""
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                sentence = []
                for word_tag in line.split():
                    if '_' not in word_tag:
                        continue  # skip malformed items
                    # Use rsplit('_',1) to handle words with underscores
                    word, tag = word_tag.rsplit('_', 1)
                    sentence.append((word, tag))
                sentences.append(sentence)
    return sentences

def train_hmm(train_data):

    transition_counts = defaultdict(lambda: defaultdict(int))
    emission_counts = defaultdict(lambda: defaultdict(int))
    tag_counts = defaultdict(int)
    vocab = set()

    all_tags = set()

    # Count occurrences
    for sentence in train_data:
        prev_tag = "<START>"
        tag_counts["<START>"] += 1

        for word, tag in sentence:
            vocab.add(word)
            all_tags.add(tag)

            emission_counts[tag][word] += 1
            tag_counts[tag] += 1

            transition_counts[prev_tag][tag] += 1
            prev_tag = tag

        transition_counts[prev_tag]["<END>"] += 1

    all_tags = list(all_tags)

    # ------------------------
    # Laplace smoothing
    # ------------------------
    V = len(vocab)
    K = len(all_tags)

    # Transition probabilities
    T = defaultdict(dict)
    for prev_tag in transition_counts:
        total = sum(transition_counts[prev_tag].values())
        for tag in all_tags + ["<END>"]:
            T[prev_tag][tag] = (transition_counts[prev_tag].get(tag, 0) + 1) / (total + K + 1)

    # START transitions: ensure smoothing
    if "<START>" not in T:
        T["<START>"] = {}
    for tag in all_tags:
        T["<START>"][tag] = (transition_counts["<START>"].get(tag, 0) + 1) / (tag_counts["<START>"] + K)

    # Emission probabilities
    E = defaultdict(dict)
    for tag in all_tags:
        total = sum(emission_counts[tag].values())
        for word in vocab:
            E[tag][word] = (emission_counts[tag].get(word, 0) + 1) / (total + V + 1)

    return T, E, all_tags, vocab


def viterbi_decode(sentence, T, E, all_tags, vocab, unk_prob=1e-6):

    N = len(sentence)
    if N == 0:
        return []

    # Viterbi probability table
    V = [{} for _ in range(N)]
    # Backpointer table
    BP = [{} for _ in range(N)]

    # ---------------------------
    # 1. Initialization
    # ---------------------------
    for tag in all_tags:

        # Emission probability with smoothing
        if sentence[0] in E[tag]:
            emit = E[tag][sentence[0]]
        else:
            emit = unk_prob
        
        # Transition from START
        trans = T["<START>"].get(tag, 0)

        V[0][tag] = trans * emit
        BP[0][tag] = None

    # If all are 0, fallback
    if sum(V[0].values()) == 0:
        return ["NN"] * N

    # ---------------------------
    # 2. Viterbi DP
    # ---------------------------
    for i in range(1, N):
        word = sentence[i]

        for tag in all_tags:

            # Emission with smoothing
            if word in E[tag]:
                emit = E[tag][word]
            else:
                emit = unk_prob

            max_prob = -1
            best_prev = None

            for prev_tag in all_tags:

                trans = T[prev_tag].get(tag, 0)
                prob = V[i-1][prev_tag] * trans * emit

                if prob > max_prob:
                    max_prob = prob
                    best_prev = prev_tag
            
            V[i][tag] = max_prob
            BP[i][tag] = best_prev

    # ---------------------------
    # 3. Backtrack best path
    # ---------------------------
    final_tag = max(V[N-1], key=V[N-1].get)

    tags = [final_tag]
    for i in range(N-1, 0, -1):
        tags.append(BP[i][tags[-1]])

    return list(reversed(tags))


# --- Evaluation Functions ---

def evaluate_tagger(true_tags, predicted_tags):
    """Calculates Precision, Recall, and F1-score."""
    
    # We use micro-averaging for a single overall score (simpler for exam)
    
    # Ensure all lists are flat and of the same length
    y_true = [tag for sublist in true_tags for tag in sublist]
    y_pred = [tag for sublist in predicted_tags for tag in sublist]
    
    if not y_true or len(y_true) != len(y_pred):
        return 0, 0, 0 # Avoid division by zero

    # Calculate overall accuracy
    correct_predictions = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    accuracy = correct_predictions / len(y_true)
    precision = accuracy
    recall = accuracy
    f1_score = accuracy

    return precision, recall, f1_score


# --- Main Execution Block ---

# a. Load data
file_path = 'wsj_pos_tagged_en.txt'
data = load_data(file_path)
print(f"Total Sentences Loaded: {len(data)}")

# a. Split the data into K folds (K >= 3)
K = 5 
kf = KFold(n_splits=K, shuffle=True, random_state=42)

all_metrics = []

for fold, (train_index, test_index) in enumerate(kf.split(data)):
    print(f"\n--- Starting Fold {fold + 1}/{K} ---")

    # Split data
    train_data = [data[i] for i in train_index]
    test_data = [data[i] for i in test_index]

    # b. Find the emission and transition probabilities from the training data.
    T, E, all_tags, vocab = train_hmm(train_data)
    print(f"Training Complete. Found {len(all_tags)} unique tags.")

    # c. Implement the Viterbi decoding and d. Evaluate the performance
    true_tags = []
    predicted_tags = []
    
    for sentence_data in test_data:
        # Extract the sequence of words for decoding
        words = [word for word, tag in sentence_data]
        # Extract the true tag sequence for evaluation
        true_tag_sequence = [tag for word, tag in sentence_data]
        
        # Predict the tag sequence
        predicted_tag_sequence = viterbi_decode(words, T, E, all_tags,vocab)
        
        true_tags.append(true_tag_sequence)
        predicted_tags.append(predicted_tag_sequence)

    # d. Evaluate the performance
    P, R, F1 = evaluate_tagger(true_tags, predicted_tags)
    all_metrics.append((P, R, F1))

    print(f"Token-level Accuracy (F1-Score): {F1 * 100:.2f}%")
    print(f"  Precision: {P:.4f}, Recall: {R:.4f}, F1-Score: {F1:.4f}")

# d. Final Summary across all folds
avg_P = np.mean([m[0] for m in all_metrics])
avg_R = np.mean([m[1] for m in all_metrics])
avg_F1 = np.mean([m[2] for m in all_metrics])

print("\n====================================")
print(f"**Final {K}-Fold Cross-Validation Results**")
print("====================================")
print(f"Average Precision: {avg_P:.4f}")
print(f"Average Recall:    {avg_R:.4f}")
print(f"Average F1-Score:  {avg_F1:.4f} (Overall Tagger Accuracy)")

Total Sentences Loaded: 3914

--- Starting Fold 1/5 ---
Training Complete. Found 45 unique tags.
Token-level Accuracy (F1-Score): 85.65%
  Precision: 0.8565, Recall: 0.8565, F1-Score: 0.8565

--- Starting Fold 2/5 ---
Training Complete. Found 45 unique tags.
Token-level Accuracy (F1-Score): 85.09%
  Precision: 0.8509, Recall: 0.8509, F1-Score: 0.8509

--- Starting Fold 3/5 ---
Training Complete. Found 44 unique tags.
Token-level Accuracy (F1-Score): 85.97%
  Precision: 0.8597, Recall: 0.8597, F1-Score: 0.8597

--- Starting Fold 4/5 ---
Training Complete. Found 45 unique tags.
Token-level Accuracy (F1-Score): 84.95%
  Precision: 0.8495, Recall: 0.8495, F1-Score: 0.8495

--- Starting Fold 5/5 ---
Training Complete. Found 45 unique tags.
Token-level Accuracy (F1-Score): 84.97%
  Precision: 0.8497, Recall: 0.8497, F1-Score: 0.8497

**Final 5-Fold Cross-Validation Results**
Average Precision: 0.8533
Average Recall:    0.8533
Average F1-Score:  0.8533 (Overall Tagger Accuracy)
